# Libraries

Installing HuggingFace Transformers (https://github.com/huggingface/transformers)

In [ ]:
!pip install transformers

import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast 
import torch

     |████████████████████████████████| 1.5MB 9.8MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
     |████████████████████████████████| 2.9MB 22.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9925cfdd1ac6977b09ae70070864ef982c0677ba96088ca7aedf90fbc5b32396
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Datset processing

Uploading the dataset, splitting the data into train and validation sets

In [ ]:
path = Path('qa_dataset.json')
data = json.loads(path.read_text(encoding='utf-8'))

train, val = train_test_split(all_data, test_size=0.2, shuffle=True)

Getting contexts, questions and answers from the the train and validation set

In [ ]:
def read_set(set):
    
    contexts = []
    questions = []
    answers = []

    for group in set:
        context = group['context']
        for qa in group['qas']:
            question = qa['question']
            for answer in qa['answers']:
                contexts.append(context)
                questions.append(question)
                answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_set(train)
val_contexts, val_questions, val_answers = read_set(val)

Adding span tags to answers and contexts

In [ ]:
def add_tags(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_tags = answer['answer_start']
        end_tags = answer['answer_end']

add_tags(train_answers, train_contexts)
add_tags(val_answers, val_contexts)

# Tokenization and vectorization

Initializating BertTokenizerFast from HuggingFace for BERT base multilingual cased pre-trained model

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

Tokenizing and vectorizing questions and contexts with BertTokenizerFast

In [ ]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

# Preparing the data for training

Adding token positions to answers

In [ ]:
def add_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_positions(train_encodings, train_answers)
add_positions(val_encodings, val_answers)

Adapting the data for training with PyTorch

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = Dataset(train_encodings)
val_dataset = Dataset(val_encodings)